In [1]:
import time
from alpaca.telescope import *
from alpaca.exceptions import *

import astropy.units as u
import numpy as np

Make sure the following programs running before continuing.
* iOptron Commander
* ASCOM Remote Server

Alternatively, for testing purposes, the Omni Simulator could also be used instead of the iOptron Commander

In [12]:
server_port = 11111
print('localhost:'+str(server_port))
T = Telescope('localhost:'+str(server_port), 0)

localhost:11111


In [13]:
if (T.Connected == True):
    print('Telescope connection is up')
else:
    print('Telescope connection is inactive')


Telescope connection is up


In [14]:
current_pointing = [T.RightAscension,T.Declination]
print('Current Telescope Pointing:', current_pointing)

Current Telescope Pointing: [0.8149194444444444, 19.157227777777777]


In [8]:
def convert_solar_coord(current_ra_dec, solar_target):
    coord_celestial = [current_ra_dec[0] * 15, current_ra_dec[1]] * u.deg  # R.A. and Dec
    position_angle = 16.697692 * u.deg
    position_angle = 0 * u.deg

    offset_arcsec = solar_target * u.arcsec # desired offset in arcseconds
    print('Desired pointing offsets [arcseconds and degrees]')
    print(offset_arcsec)
    print((offset_arcsec.to(u.deg)))

    # Equation 25
    offset_arcsec_sin = np.sin(offset_arcsec)
    offset_arcsec_cos = np.cos(offset_arcsec)
    offset_arcsec_tan = np.tan(offset_arcsec)

    ratio_pos_angle = -offset_arcsec_sin[0]/offset_arcsec_tan[1]

    ratio_pos_angle_atan = np.arctan(ratio_pos_angle)

    pos_angle = ratio_pos_angle_atan + position_angle

    # Equation 26
    sum_offsets = np.sqrt(offset_arcsec_sin[0]**2 + offset_arcsec_tan[1]**2) / offset_arcsec_cos[0]
    angular_distance = np.arctan(sum_offsets)

    #print('Desired pointing offsets [position angle, angular distance]')
    #print(pos_angle.to(u.deg), angular_distance.to(u.deg))

    # Equation 27
    sin_declination = np.sin(coord_celestial[1]) * np.cos(angular_distance) + \
                      np.cos(coord_celestial[1]) * np.sin(angular_distance) * np.cos(pos_angle)
    declination     = np.arcsin(sin_declination).to(u.deg)

    #print(declination.to(u.deg)

    # Equation 28
    tan_ra_offset =  np.sin(angular_distance) * np.sin(pos_angle) / \
                    (np.cos(angular_distance) * np.cos(coord_celestial[1]) - \
                     np.sin(angular_distance) * np.sin(coord_celestial[1]) * np.cos(pos_angle))
    ra_offset = np.arctan(tan_ra_offset)

    print('R.A. and Dec offsets')
    print(ra_offset.to(u.deg),declination.to(u.deg) - coord_celestial[1])
    
    return((coord_celestial[0] + ra_offset)/15, declination)

In [31]:
print(current_pointing)
new_target_pointing = convert_solar_coord(current_pointing, [900,0])
print(new_target_pointing[0])
print(new_target_pointing[1])

[0.8149194444444444, 19.157227777777777]
Desired pointing offsets [arcseconds and degrees]
[900.   0.] arcsec
[0.25 0.  ] deg
R.A. and Dec offsets
-0.25360716641260844 deg 0.07165636419083654 deg
0.8318265888719517 deg
19.228884141968614 deg


In [7]:
repoint = T.SlewToCoordinatesAsync(new_target_pointing[0].value, new_target_pointing[1].value)

In [8]:
print('Current Telescope Pointing:', T.RightAscension,T.Declination)

Current Telescope Pointing: 23.9719 19.90306111111111


In [9]:
def go_to_solar(solar_target, current_solar=[0,0]):
    current_pointing = [T.RightAscension,T.Declination]
    print('Telescope pointing before move:', current_pointing[0],current_pointing[1])
    
    combined_offset = [0,0]
    combined_offset[0] = solar_target[0]- current_solar[0]
    combined_offset[1] = solar_target[1]- current_solar[1]
    print('Combined Offsets:', combined_offset[0],combined_offset[1])

    new_target_pointing = convert_solar_coord(current_pointing, combined_offset)

    repoint = T.SlewToCoordinatesAsync(new_target_pointing[0].value, new_target_pointing[1].value)
    while(T.Slewing):
        time.sleep(0.5)               

    print('Current Telescope Pointing:', T.RightAscension,T.Declination)
    

In [10]:
go_to_solar([-357,864], [300,300])

Telescope pointing before move: 23.97190277777778 19.90306111111111
Combined Offsets: -657 564
Desired pointing offsets [arcseconds and degrees]
[-657.  564.] arcsec
[-0.1825      0.15666667] deg
R.A. and Dec offsets
0.2339263207783043 deg 0.0974717996865202 deg
Current Telescope Pointing: 23.987513888888888 20.000544444444444


In [15]:
print('Current Telescope Pointing:', T.RightAscension,T.Declination)

Current Telescope Pointing: 0.8149166666666667 19.157227777777777


In [26]:
go_to_solar([-200,0],[0,0])

Telescope pointing before move: 0.8149333333333334 19.15725
Combined Offsets: -200 0
Desired pointing offsets [arcseconds and degrees]
[-200.    0.] arcsec
[-0.05555556  0.        ] deg
R.A. and Dec offsets
0.05632716873599322 deg -0.015970911141522492 deg
Current Telescope Pointing: 0.8187027777777778 19.1413


In [27]:
repoint = T.SlewToCoordinatesAsync(0.8149194444444444, 19.157227777777777)

In [10]:
coord_new = convert_solar_coord([19.055273, -22.6386],[960,0])

Desired pointing offsets [arcseconds and degrees]
[960.   0.] arcsec
[0.26666667 0.        ] deg
R.A. and Dec offsets
-0.2889279065508774 deg 0.00025880476302120314 deg


In [14]:
print(coord_new,(coord_new[0].value - int(coord_new[0].value))*60)

(<Quantity 19.03601114 deg>, <Quantity -22.6383412 deg>) 2.160668373796497
